In [1]:
from __future__ import print_function, division

from keras.datasets import mnist
from keras.layers import Input, Dense, Reshape, Flatten, Dropout
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import Adam
import keras.backend as K

import matplotlib.pyplot as plt

import sys

import numpy as np
import os
class BGAN():
    """Reference: https://wiseodd.github.io/techblog/2017/03/07/boundary-seeking-gan/"""
    def __init__(self):
        self.img_rows = 28
        self.img_cols = 28
        self.channels = 1
        self.img_shape = (self.img_rows, self.img_cols, self.channels)
        self.latent_dim = 100

        optimizer = Adam(0.0002, 0.5)

        # Build and compile the discriminator
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss='binary_crossentropy',
            optimizer=optimizer,
            metrics=['accuracy'])

        # Build the generator
        self.generator = self.build_generator()

        # The generator takes noise as input and generated imgs
        z = Input(shape=(self.latent_dim,))
        img = self.generator(z)

        # For the combined model we will only train the generator
        self.discriminator.trainable = False

        # The valid takes generated images as input and determines validity
        valid = self.discriminator(img)

        # The combined model  (stacked generator and discriminator)
        # Trains the generator to fool the discriminator
        self.combined = Model(z, valid)
        self.combined.compile(loss=self.boundary_loss, optimizer=optimizer)

    def build_generator(self):

        model = Sequential()

        model.add(Dense(256, input_dim=self.latent_dim))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(1024))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(np.prod(self.img_shape), activation='tanh'))
        model.add(Reshape(self.img_shape))

        model.summary()

        noise = Input(shape=(self.latent_dim,))
        img = model(noise)

        return Model(noise, img)

    def build_discriminator(self):

        model = Sequential()

        model.add(Flatten(input_shape=self.img_shape))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(256))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(1, activation='sigmoid'))
        model.summary()

        img = Input(shape=self.img_shape)
        validity = model(img)

        return Model(img, validity)

    def boundary_loss(self, y_true, y_pred):
        """
        Boundary seeking loss.
        Reference: https://wiseodd.github.io/techblog/2017/03/07/boundary-seeking-gan/
        """
        return 0.5 * K.mean((K.log(y_pred) - K.log(1 - y_pred))**2)

    def train(self, epochs, batch_size=128, sample_interval=50):

        # Load the dataset
        (X_train, _), (_, _) = mnist.load_data()

        # Rescale -1 to 1
        X_train = X_train / 127.5 - 1.
        X_train = np.expand_dims(X_train, axis=3)

        # Adversarial ground truths
        valid = np.ones((batch_size, 1))
        fake = np.zeros((batch_size, 1))
        nb_batches = int(X_train.shape[0] / batch_size)
        global_step = 0

        for epoch in range(epochs):

            # ---------------------
            #  Train Discriminator
            # ---------------------

            # Select a random batch of images
            # idx = np.random.randint(0, X_train.shape[0], batch_size)
            for index in range(nb_batches):
                global_step += 1
                # progress_bar.update(index)

                # get a batch of real images
                image_batch = X_train[index * batch_size:(index + 1) * batch_size]

                noise = np.random.normal(0, 1, (batch_size, self.latent_dim))

                # Generate a batch of new images
                gen_imgs = self.generator.predict(noise)

                # Train the discriminator
                d_loss_real = self.discriminator.train_on_batch(image_batch, valid)
                d_loss_fake = self.discriminator.train_on_batch(gen_imgs, fake)
                d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

                # ---------------------
                #  Train Generator
                # ---------------------

                g_loss = self.combined.train_on_batch(noise, valid)

                # Plot the progress
                print("epoch:%d step:%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch,global_step, d_loss[0], 100 * d_loss[1], g_loss))

                # If at save interval => save generated image samples
                if global_step % sample_interval == 0:
                    self.sample_images(epoch,global_step)


    def sample_images(self, epoch,global_step):
        r, c = 10, 10
        noise = np.random.normal(0, 1, (r * c, self.latent_dim))
        gen_imgs = self.generator.predict(noise)
        # Rescale images 0 - 1
        gen_imgs = 0.5 * gen_imgs + 0.5

        fig, axs = plt.subplots(r, c)
        cnt = 0
        for i in range(r):
            for j in range(c):
                axs[i,j].imshow(gen_imgs[cnt, :,:,0], cmap='gray')
                axs[i,j].axis('off')
                cnt += 1
        if not os.path.isdir('images_bgan'):
            os.mkdir('images_bgan')
        fig.savefig("images_bgan/epoch_%d_step_%d.png" % (epoch,global_step))
        plt.close()


if __name__ == '__main__':
    bgan = BGAN()
    bgan.train(epochs=30, batch_size=64, sample_interval=200)


Using TensorFlow backend.


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1 (Flatten)          (None, 784)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               401920    
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 256)               131328    
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 256)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 257       
Total params: 533,505
Trainable params: 533,505
Non-trainable params: 0
_________________________________________________________________
____

/home/imi432_006/anaconda3/envs/tf/lib/python3.5/site-packages/keras/engine/training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


epoch:0 step:1 [D loss: 0.986237, acc.: 3.12%] [G loss: 0.146112]
epoch:0 step:2 [D loss: 0.472062, acc.: 69.53%] [G loss: 0.074949]
epoch:0 step:3 [D loss: 0.348500, acc.: 81.25%] [G loss: 0.131366]
epoch:0 step:4 [D loss: 0.322515, acc.: 85.16%] [G loss: 0.248839]
epoch:0 step:5 [D loss: 0.241025, acc.: 98.44%] [G loss: 0.366994]
epoch:0 step:6 [D loss: 0.215772, acc.: 99.22%] [G loss: 0.523038]
epoch:0 step:7 [D loss: 0.190584, acc.: 99.22%] [G loss: 0.674367]
epoch:0 step:8 [D loss: 0.168803, acc.: 99.22%] [G loss: 0.726735]
epoch:0 step:9 [D loss: 0.155547, acc.: 99.22%] [G loss: 0.941902]
epoch:0 step:10 [D loss: 0.151439, acc.: 100.00%] [G loss: 0.992103]
epoch:0 step:11 [D loss: 0.134715, acc.: 100.00%] [G loss: 1.214959]
epoch:0 step:12 [D loss: 0.138540, acc.: 100.00%] [G loss: 1.314952]
epoch:0 step:13 [D loss: 0.118236, acc.: 100.00%] [G loss: 1.465600]
epoch:0 step:14 [D loss: 0.114718, acc.: 100.00%] [G loss: 1.557668]
epoch:0 step:15 [D loss: 0.112591, acc.: 100.00%] [G 

epoch:0 step:120 [D loss: 0.440543, acc.: 74.22%] [G loss: 6.337528]
epoch:0 step:121 [D loss: 0.051268, acc.: 100.00%] [G loss: 9.072118]
epoch:0 step:122 [D loss: 0.934221, acc.: 68.75%] [G loss: 7.722188]
epoch:0 step:123 [D loss: 0.221105, acc.: 91.41%] [G loss: 5.233693]
epoch:0 step:124 [D loss: 0.090721, acc.: 98.44%] [G loss: 6.306021]
epoch:0 step:125 [D loss: 0.091072, acc.: 99.22%] [G loss: 6.348392]
epoch:0 step:126 [D loss: 0.095613, acc.: 99.22%] [G loss: 6.517221]
epoch:0 step:127 [D loss: 0.111776, acc.: 98.44%] [G loss: 5.727079]
epoch:0 step:128 [D loss: 0.101437, acc.: 97.66%] [G loss: 6.290627]
epoch:0 step:129 [D loss: 0.107292, acc.: 98.44%] [G loss: 6.048798]
epoch:0 step:130 [D loss: 0.131319, acc.: 97.66%] [G loss: 6.135802]
epoch:0 step:131 [D loss: 0.110547, acc.: 97.66%] [G loss: 7.130000]
epoch:0 step:132 [D loss: 0.190573, acc.: 94.53%] [G loss: 3.173328]
epoch:0 step:133 [D loss: 0.104270, acc.: 98.44%] [G loss: 5.165356]
epoch:0 step:134 [D loss: 0.07256

...

epoch:29 step:27599 [D loss: 0.441641, acc.: 80.47%] [G loss: 0.928314]
epoch:29 step:27600 [D loss: 0.418387, acc.: 81.25%] [G loss: 1.117514]
epoch:29 step:27601 [D loss: 0.502750, acc.: 75.00%] [G loss: 0.807177]
epoch:29 step:27602 [D loss: 0.428889, acc.: 79.69%] [G loss: 1.101907]
epoch:29 step:27603 [D loss: 0.470315, acc.: 77.34%] [G loss: 0.928697]
epoch:29 step:27604 [D loss: 0.510597, acc.: 71.88%] [G loss: 0.706391]
epoch:29 step:27605 [D loss: 0.545740, acc.: 69.53%] [G loss: 0.787190]
epoch:29 step:27606 [D loss: 0.494961, acc.: 72.66%] [G loss: 0.790927]
epoch:29 step:27607 [D loss: 0.502234, acc.: 74.22%] [G loss: 0.697735]
epoch:29 step:27608 [D loss: 0.527646, acc.: 67.97%] [G loss: 0.826253]
epoch:29 step:27609 [D loss: 0.458363, acc.: 74.22%] [G loss: 0.912994]
epoch:29 step:27610 [D loss: 0.663972, acc.: 59.38%] [G loss: 0.625465]
epoch:29 step:27611 [D loss: 0.578889, acc.: 64.84%] [G loss: 0.712283]
epoch:29 step:27612 [D loss: 0.523997, acc.: 75.78%] [G loss: 0.

epoch:29 step:27717 [D loss: 0.558148, acc.: 71.88%] [G loss: 0.684731]
epoch:29 step:27718 [D loss: 0.554824, acc.: 71.09%] [G loss: 0.813334]
epoch:29 step:27719 [D loss: 0.457913, acc.: 78.91%] [G loss: 0.913282]
epoch:29 step:27720 [D loss: 0.555941, acc.: 67.19%] [G loss: 0.962627]
epoch:29 step:27721 [D loss: 0.503335, acc.: 75.78%] [G loss: 0.961106]
epoch:29 step:27722 [D loss: 0.482206, acc.: 75.00%] [G loss: 0.793044]
epoch:29 step:27723 [D loss: 0.475389, acc.: 74.22%] [G loss: 0.776832]
epoch:29 step:27724 [D loss: 0.466946, acc.: 78.12%] [G loss: 0.798258]
epoch:29 step:27725 [D loss: 0.468547, acc.: 77.34%] [G loss: 0.725733]
epoch:29 step:27726 [D loss: 0.568672, acc.: 70.31%] [G loss: 0.768910]
epoch:29 step:27727 [D loss: 0.434442, acc.: 78.12%] [G loss: 1.039485]
epoch:29 step:27728 [D loss: 0.428037, acc.: 78.12%] [G loss: 0.948554]
epoch:29 step:27729 [D loss: 0.534732, acc.: 73.44%] [G loss: 0.887731]
epoch:29 step:27730 [D loss: 0.513956, acc.: 75.00%] [G loss: 0.

epoch:29 step:27831 [D loss: 0.519780, acc.: 70.31%] [G loss: 0.821565]
epoch:29 step:27832 [D loss: 0.513206, acc.: 72.66%] [G loss: 0.821273]
epoch:29 step:27833 [D loss: 0.498527, acc.: 75.00%] [G loss: 0.902377]
epoch:29 step:27834 [D loss: 0.499254, acc.: 73.44%] [G loss: 0.915270]
epoch:29 step:27835 [D loss: 0.445936, acc.: 75.78%] [G loss: 0.896988]
epoch:29 step:27836 [D loss: 0.514795, acc.: 71.88%] [G loss: 0.712216]
epoch:29 step:27837 [D loss: 0.564200, acc.: 65.62%] [G loss: 0.929595]
epoch:29 step:27838 [D loss: 0.573253, acc.: 69.53%] [G loss: 0.739696]
epoch:29 step:27839 [D loss: 0.449615, acc.: 81.25%] [G loss: 0.781886]
epoch:29 step:27840 [D loss: 0.520236, acc.: 77.34%] [G loss: 0.793285]
epoch:29 step:27841 [D loss: 0.619246, acc.: 61.72%] [G loss: 0.618081]
epoch:29 step:27842 [D loss: 0.510580, acc.: 75.00%] [G loss: 0.748386]
epoch:29 step:27843 [D loss: 0.566728, acc.: 71.09%] [G loss: 0.592248]
epoch:29 step:27844 [D loss: 0.581588, acc.: 67.97%] [G loss: 0.

epoch:29 step:27952 [D loss: 0.534866, acc.: 73.44%] [G loss: 0.790321]
epoch:29 step:27953 [D loss: 0.428416, acc.: 79.69%] [G loss: 0.761788]
epoch:29 step:27954 [D loss: 0.454136, acc.: 75.00%] [G loss: 0.809543]
epoch:29 step:27955 [D loss: 0.460669, acc.: 78.12%] [G loss: 1.098123]
epoch:29 step:27956 [D loss: 0.482582, acc.: 75.78%] [G loss: 1.332481]
epoch:29 step:27957 [D loss: 0.587496, acc.: 70.31%] [G loss: 0.890623]
epoch:29 step:27958 [D loss: 0.500198, acc.: 78.12%] [G loss: 0.703218]
epoch:29 step:27959 [D loss: 0.496747, acc.: 75.78%] [G loss: 0.794322]
epoch:29 step:27960 [D loss: 0.559474, acc.: 70.31%] [G loss: 0.783280]
epoch:29 step:27961 [D loss: 0.582156, acc.: 70.31%] [G loss: 0.814230]
epoch:29 step:27962 [D loss: 0.512605, acc.: 75.00%] [G loss: 0.758502]
epoch:29 step:27963 [D loss: 0.478396, acc.: 73.44%] [G loss: 0.747051]
epoch:29 step:27964 [D loss: 0.574365, acc.: 71.09%] [G loss: 0.713365]
epoch:29 step:27965 [D loss: 0.463402, acc.: 71.88%] [G loss: 0.

epoch:29 step:28071 [D loss: 0.388833, acc.: 83.59%] [G loss: 0.980645]
epoch:29 step:28072 [D loss: 0.437649, acc.: 79.69%] [G loss: 0.982872]
epoch:29 step:28073 [D loss: 0.454860, acc.: 82.03%] [G loss: 0.807591]
epoch:29 step:28074 [D loss: 0.566378, acc.: 71.09%] [G loss: 0.776410]
epoch:29 step:28075 [D loss: 0.617441, acc.: 64.84%] [G loss: 0.654344]
epoch:29 step:28076 [D loss: 0.522473, acc.: 69.53%] [G loss: 0.728431]
epoch:29 step:28077 [D loss: 0.583220, acc.: 71.09%] [G loss: 0.720243]
epoch:29 step:28078 [D loss: 0.560016, acc.: 67.97%] [G loss: 0.663932]
epoch:29 step:28079 [D loss: 0.446026, acc.: 82.03%] [G loss: 0.849584]
epoch:29 step:28080 [D loss: 0.524890, acc.: 72.66%] [G loss: 0.705915]
epoch:29 step:28081 [D loss: 0.565975, acc.: 66.41%] [G loss: 0.673951]
epoch:29 step:28082 [D loss: 0.456006, acc.: 75.00%] [G loss: 0.987622]
epoch:29 step:28083 [D loss: 0.565168, acc.: 70.31%] [G loss: 0.707872]
epoch:29 step:28084 [D loss: 0.486660, acc.: 75.00%] [G loss: 0.